## CIFAR 10

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.conv_learner import *
from fastai.column_data import *
from fastai.structured import *
import pandas as pd
import cv2
#PATH = "data/cifar10/"
PATH = "data/iceberg/"
os.makedirs(PATH,exist_ok=True)

In [3]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))

In [4]:
#Load the data.
# TODO: write 'train' and 'valid' to files to avoid confusing variables
train = pd.read_json(f'{PATH}train.json') 
test = pd.read_json(f'{PATH}test.json')

In [5]:
train.head()

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0


In [5]:
train_df = train.copy()
train_df = train_df.set_index("id")

In [7]:
train_df.head()

,band_1,band_2,inc_angle,is_iceberg
id,,,,
dfd5f913,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",43.9239,0
e25388fd,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",38.1562,0
58b2aaa0,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",45.2859,1
4cfc3a18,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",43.8306,0
271f93f4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",35.6256,0


In [13]:
train_df.reset_index

In [6]:
y = train_df['is_iceberg'].values
y[:5]

array([0, 0, 1, 0, 0])

In [21]:
train_df.head()

,id,band_1,band_2,inc_angle,is_iceberg
0,dfd5f913,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",43.9239,0
1,e25388fd,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",38.1562,0
2,58b2aaa0,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",45.2859,1
3,4cfc3a18,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",43.8306,0
4,271f93f4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",35.6256,0


In [30]:
train_df.index -= 1

In [31]:
train_df.head()

,id,band_1,band_2,inc_angle,is_iceberg
0,dfd5f913,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",43.9239,0
1,e25388fd,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",38.1562,0
2,58b2aaa0,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",45.2859,1
3,4cfc3a18,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",43.8306,0
4,271f93f4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",35.6256,0


In [44]:
val_idx = np.array(train_df.index.values)
#df = train
cat_vars = None
x1 = np.array(train['band_1'])
x2 = np.array(train['band_2'])
df_test = test
y = np.array(train['is_iceberg'])

In [54]:
md = ColumnarModelData.from_arrays(PATH, val_idx, x1,x2, y)

In [ ]:
m = md.get_learner

In [57]:
def inv_y(a): return np.exp(a)

def exp_rmspe(y_pred, targ):
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))/targ
    return math.sqrt((pct_var**2).mean())

max_log_y = np.max(y)
y_range = (0, max_log_y*1.2)

In [58]:
m = md.get_learner(0, len(train.columns),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-3

TypeError: 'int' object is not iterable

In [32]:
# create train and validation sets
from sklearn.model_selection import train_test_split #TODO: if successful, move to utils.py
train, valid = train_test_split(train, test_size = 0.08)

In [33]:
print(train.shape)
print(valid.shape)
print(test.shape)

(1475, 5)
(129, 5)
(8424, 4)


In [54]:
a = np.array([1, 2, 3, 4])
b = np.array([5,6,7,8])
c = np.concatenate([a[:, np.newaxis],
                    b[:, np.newaxis],
                    ((a+b)/2)[:, np.newaxis]], axis=-1)
d = np.dstack([a,
                b,
                (a+b)/2])
print(a)
print (d)
c.reshape(4,3)

[1 2 3 4]
[[[ 1.  5.  3.]
  [ 2.  6.  4.]
  [ 3.  7.  5.]
  [ 4.  8.  6.]]]


array([[ 1.,  5.,  3.],
       [ 2.,  6.,  4.],
       [ 3.,  7.,  5.],
       [ 4.,  8.,  6.]])

In [40]:
#Generate the training data
#Create 3 bands having HH, HV and avg of both
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train["band_2"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], 
                          X_band_2[:, :, :, np.newaxis],
                         ((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)
Y_train = np.array(train["is_iceberg"])

In [41]:
X_train.shape

(1475, 75, 75, 3)

In [35]:
#Generate the validation data
#Create 3 bands having HH, HV and avg of both
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in valid["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in valid["band_2"]])
X_valid = np.concatenate([X_band_1[:, :, :, np.newaxis], 
                          X_band_2[:, :, :, np.newaxis],
                          ((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)
Y_valid = np.array(valid["is_iceberg"])

In [36]:
X_valid.shape

(129, 75, 75, 3)

In [55]:
plt.imsave(X_valid-0, X_valid[0], format='tif')

ValueError: Floating point image RGB values must be in the 0..1 range.

In [38]:
print(X_train.shape)
print(Y_train.shape)
print(X_valid.shape)
print(Y_valid.shape)

(1475, 75, 75, 3)
(1475,)
(129, 75, 75, 3)
(129,)


In [42]:
X_band_1.shape

(1475, 75, 75)

In [ ]:
# first look at an image; this one is a ship (I believe)
plt.imshow(X_band_1[31], interpolation='nearest');

In [ ]:
# plot method taken from Kaggle kernel example
#Take a look at a ship
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
def plotmy3d(c, name):

    data = [
        go.Surface(
            z=c
        )
    ]
    layout = go.Layout(
        title=name,
        autosize=False,
        width=700,
        height=700,
        margin=dict(
            l=65,
            r=50,
            b=65,
            t=90
        )
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)
plotmy3d(X_band_1[31,:,:], 'ship')

In [ ]:
def get_data(sz,bs):
    tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomFlipXY()], pad=sz//8)
    return ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs)

In [43]:
def get_data_arrays(sz,bs, X_train, Y_train, X_valid, Y_valid):
    tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomFlipXY()], pad=sz//8)
    return ImageClassifierData.from_arrays(PATH, (X_train,Y_train), (X_valid, Y_valid), tfms=tfms, bs=bs)

In [9]:
??ImageClassifierData

In [44]:
bs=128

### Look at data

In [45]:
data = get_data_arrays(32,4, X_train, Y_train, X_valid, Y_valid)

In [59]:
x,y=next(iter(data.trn_dl))

In [ ]:
plt.imshow(data.trn_ds.denorm(x));


In [ ]:
plt.imshow(data.trn_ds.denorm(x)[1]);

## Initial model

In [70]:
from fastai.models.cifar10.resnext import resnext29_8_64
#from fastai.models.cifar10.senet import SENet34

m = resnext29_8_64()
#m = SENet34()

bm = BasicModel(m.cuda(), name='cifar10_rn29_8_64')
#bm = BasicModel(m.cuda(), name='cifar10_senet34')

In [71]:
data = get_data_arrays(8,bs*4, X_train, Y_train, X_valid, Y_valid)

In [72]:
learn = ConvLearner(data, bm)
learn.unfreeze()

In [73]:
lr=1e-2; wd=5e-4

In [74]:
learn.lr_find

<bound method Learner.lr_find of CifarResNeXt(
  (conv_1_3x3): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (stage_1): Sequential(
    (0): ResNeXtBottleneck(
      (downsample): Sequential(
        (0): Conv2d (64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      )
      (conv_reduce): Conv2d (64, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn_reduce): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
      (conv_conv): Conv2d (512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=8, bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
      (conv_expand): Conv2d (512, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn_expand): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): ResNeXtBottleneck(
      (conv_reduce): Conv2d

In [53]:
learn.sched.plot()

AttributeError: 'NoneType' object has no attribute 'plot'

In [ ]:
%time learn.fit(lr, 1)

In [ ]:
learn.fit(lr, 2, cycle_len=1)

In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2, wds=wd)

In [ ]:
learn.save('8x8_8')

## 16x16

In [ ]:
learn.load('8x8_8')

In [ ]:
%cd fastai/courses/dl1

In [ ]:
learn.load('sen_32x32_8')

In [ ]:
learn.set_data(get_data(16,bs*2))

In [ ]:
%time learn.fit(1e-3, 1, wds=wd)

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.sched.plot()

In [ ]:
lr=1e-3

In [ ]:
learn.fit(lr, 2, cycle_len=1, wds=wd)

In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2, wds=wd)

In [ ]:
learn.save('sen_32x32_9')

## 24x24

In [ ]:
learn.load('sen_32x32_9')

In [ ]:
learn.set_data(get_data(24,bs))

In [ ]:
learn.fit(1e-2, 1, wds=wd)

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit(lr, 1, cycle_len=1, wds=wd)

In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2, wds=wd)

In [ ]:
learn.save('24x24_8')

In [ ]:
log_preds,y = learn.TTA()
preds = np.mean(np.exp(log_preds),0)metrics.log_loss(y,preds), accuracy(preds,y)

## 32x32

In [ ]:
learn.load('24x24_8')

In [ ]:
learn.set_data(get_data(32,bs))

In [ ]:
learn.fit(1e-2, 1, wds=wd)

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2, wds=wd)

In [ ]:
learn.fit(lr, 3, cycle_len=4, wds=wd)

In [ ]:
log_preds,y = learn.TTA()
metrics.log_loss(y,np.exp(log_preds)), accuracy(log_preds,y)

In [ ]:
learn.save('32x32_8')